In [9]:
import yaml
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
import json
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, UnstructuredHTMLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from typing import List
from langchain_core.documents import Document
import os
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict


load_dotenv()

# Load Prompt from YAML
with open("prompt.yaml", "r") as f:
    prompts = yaml.safe_load(f)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
print(prompts)

{'agent_prompt': 'You have access to multiple healthcare-related JSON data sources. Each source contains detailed patient information as described below:\n\n1. claims:\n  Contains healthcare claim details submitted for medical services.\n  - claimId: Unique identifier of the claim.\n  - date: Date the claim was filed.\n  - provider: Healthcare provider (doctor/hospital), anonymized as "RESTRICTED".\n  - status: Approval status of the claim (e.g., Approved, Denied).\n  - diagnosis: Primary diagnosis associated with the claim.\n  - charges: Total amount billed.\n  - paid: Amount paid by insurance.\n  - claimProvider: Specific provider who submitted the claim.\n  - problems: List of diagnoses/problems associated with the claim.\n  - services: Services billed in this claim.\n\n2. communications:\n  Contains patient-provider communications.\n  - date/contactDate: Date of communication.\n  - source: Origin/source of communication.\n  - callDirection: Direction of call (incoming/outgoing).\n 

In [10]:
llm = init_chat_model("gpt-4o", model_provider="openai")

In [11]:
llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BORtAkv9jb95c9GKKqLk241MuwjKo', 'finish_reason': 'stop', 'logprobs': None}, id='run-db5224bc-3b89-4ba4-9dde-88eaa304bc46-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
llm.invoke("What is the status of my recent claim?")

AIMessage(content="I'm unable to access personal data or accounts, including information about the status of any claims. For updates on your recent claim, please contact the relevant company or organization directly through their customer service or online portal.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 16, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BORtRL1GtqO6x5TTxVFPnh7OFgmwg', 'finish_reason': 'stop', 'logprobs': None}, id='run-22e40a86-0c5d-431e-8148-9160cee674b6-0', usage_metadata={'input_tokens': 16, 'output_tokens': 42, 'total_tokens': 58, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reaso

In [16]:
message = {}
message["System Message"] = prompts
message["User"] = "HI"

In [17]:
message

{'System Message': {'agent_prompt': 'You have access to multiple healthcare-related JSON data sources. Each source contains detailed patient information as described below:\n\n1. claims:\n  Contains healthcare claim details submitted for medical services.\n  - claimId: Unique identifier of the claim.\n  - date: Date the claim was filed.\n  - provider: Healthcare provider (doctor/hospital), anonymized as "RESTRICTED".\n  - status: Approval status of the claim (e.g., Approved, Denied).\n  - diagnosis: Primary diagnosis associated with the claim.\n  - charges: Total amount billed.\n  - paid: Amount paid by insurance.\n  - claimProvider: Specific provider who submitted the claim.\n  - problems: List of diagnoses/problems associated with the claim.\n  - services: Services billed in this claim.\n\n2. communications:\n  Contains patient-provider communications.\n  - date/contactDate: Date of communication.\n  - source: Origin/source of communication.\n  - callDirection: Direction of call (inc

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model

# Step 1: Initialize LLM
llm = init_chat_model("gpt-4o", model_provider="openai")

# Step 2: Extract string from dictionary
prompt_text = prompts["agent_prompt"]  # <- fix: don't pass full dict

# Step 3: Create prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_text),
    ("human", "{input}")
])

# Step 4: Chain and test
chain = prompt | llm | StrOutputParser()

# Step 5: Try it
response = chain.invoke({"input": "any pcp visit after er?"})
print(response)


["emergency_dept", "office_visits"]


In [25]:
response = chain.invoke({"input": "What was my recent discharge date?"})
print(response)

["inPatient_records"]


In [26]:
response = chain.invoke({"input": "How are you?"})
print(response)

I'm just a program, but I'm here to help you with your healthcare data queries. How can I assist you today?


In [ ]:
import yaml
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from tools import retrieve
import json

# Load Prompt from YAML
with open("prompt.yaml", "r") as f:
    prompts = yaml.safe_load(f)

# Initialize the LLM
llm = init_chat_model("gpt-4o", model_provider="openai")

# Define Prompt Template
prompt_template = ChatPromptTemplate.from_template(prompts["agent_prompt"])

# Define the Agent Node (LLM)
def agent(state):
    messages = state["messages"]
    response = llm.invoke(messages)
    content = response.content.strip()

    # Check if action is needed (JSON array of sources)
    try:
        sources = json.loads(content)
        if isinstance(sources, list):
            return {"sources": sources, "messages": messages}
    except json.JSONDecodeError:
        pass

    # Otherwise, direct response
    messages.append(AIMessage(content=content))
    return {"messages": messages}

# Define the Tool Retrieval Node
def retrieval_node(state):
    sources = state["sources"]
    retrieval_result = retrieve.invoke(sources)
    messages = state["messages"]
    messages.append(HumanMessage(content=retrieval_result))
    return {"messages": messages}

# State Graph Setup
graph = StateGraph(schema={"messages": list, "sources": list})

# Nodes setup
graph.add_node("agent", agent)
graph.add_node("retriever", retrieval_node)

# Edges & Logic
graph.set_entry_point("agent")

# Routing logic
def route(state):
    if "sources" in state:
        return "retriever"
    return END

graph.add_conditional_edges("agent", route, {"retriever": "retriever", END: END})
graph.add_edge("retriever", END)

# Compile the graph
app = graph.compile()

# Function to interact with the agent
def chat_with_agent(query):
    inputs = {"messages": [HumanMessage(content=query)]}
    final_state = app.invoke(inputs)
    messages = final_state["messages"]
    for msg in messages:
        if isinstance(msg, AIMessage):
            print(f"Assistant:\n{msg.content}\n")

# Example interactions:
if __name__ == "__main__":
    # Conversational example
    chat_with_agent("Hello!")

    # Retrieval example
    chat_with_agent("What's the status of my recent claim?")
